In [ ]:
import numpy as np
import pandas as pd

In [ ]:
batting_path = 'Batting.csv'
people_path = 'People.csv'
team_path = 'Teams.csv'
salaries_path = 'Salaries.csv'
app_path = 'Appearances.csv'
batting = pd.read_csv(batting_path)
players = pd.read_csv(people_path)
teams = pd.read_csv(team_path)
money = pd.read_csv(salaries_path)
pos = pd.read_csv(app_path)

In [ ]:
teams.columns

In [ ]:
# display head of batting
batting.head(10)

In [ ]:
# print the columns of each dataframe
print('batting: ', batting.columns)
print('player: ', players.columns)
print('teams: ', teams.columns)
print('money: ', money.columns)

In [ ]:
# print summary statistics for batting
teams.describe()

In [ ]:
# look at team ids (unique values)
teams['name'].unique()
teams['name'].nunique()

In [ ]:
# look for missing values in each of the dataframes

batting.isnull().sum()
batting.query('yearID>1955').isnull().sum()
batting[(batting['yearID'] > 1950) & (batting['IBB'].isnull())]

In [ ]:
# look for duplicate values in batting

batting.duplicated().sum()
batting.duplicated(subset=['playerID', 'yearID']).sum()
batting.duplicated(subset=['playerID', 'yearID', 'stint']).sum()

In [ ]:
# look into stint

batting.sort_values('stint', ascending=False)


loc1 = batting['playerID'] == 'drakeol01'
loc2 = batting['yearID'] == 2018

batting[loc1 & loc2]

batting.groupby(['playerID', 'yearID', 'teamID'])['stint'].count().sort_values()


loc3 = batting['playerID'] == 'chouife01'
loc4 = batting['yearID'] == 1914


batting[loc3 & loc4]

In [ ]:
# filter the df_batting such that it only contains yearID = 1990
# filter df_batting such that it only contains yearID = 1991

bat1990 = batting[batting['yearID'] == 1990]
bat1991 = batting[batting['yearID'] == 1991]

bat1990sum = batting[batting['yearID'] == 1990].groupby(['playerID', 'yearID']).sum()
bat1991sum = batting[batting['yearID'] == 1991].groupby(['playerID', 'yearID']).sum()


bat1990sum + bat1991sum

#why doesn't this work???

#two ways



In [ ]:
## correct answer
bat1990sum.reset_index().drop('yearID', axis=1).set_index('playerID') + \
    bat1991sum.reset_index().drop('yearID', axis=1).set_index('playerID')


In [ ]:
# look at players dataframe


players.head(10)
players.isnull().sum()

players[players['debut'].isnull()]

In [ ]:
#look at the teams dataframe

teams.head(10)

teams.isnull().sum()

teams[teams['divID'].isnull()]

In [ ]:
# in the batting dataframe, group the rows by playerID and yearID
# (they may have played for more than 1 team)
# calculate their total number of at-bats (AB), H
# doubles (2B), triples (3B), home runs(HR), and walks
# copy this to a new dataframe

bat = batting[batting['yearID']> 1912]

bat = bat.groupby(['playerID', 'yearID']).sum()
cols = ['AB', 'H', '2B', '3B', 'HR', 'BB', 'R']

bat = bat[cols]

In [ ]:
# calculate each player's batting average: H/AB
# calculate each player's on-base-percentage: (H+BB) / (AB+BB)
# calculate the number of single (1B) a player hit:
# H - 2B - 3B - HR
# calculate a player's slugging percentage:
# (1B + 2 * 2B + 3* 3B + 4* HR) / (AB)


bat['BA'] = bat['H'] / bat['AB']
bat['BA'] = bat['BA'].fillna(0.0)

bat['OBP'] = (bat['H'] + bat['BB']) / (bat['AB'] + bat['BB'])
bat['OBP'].fillna(0.0, inplace=True)

bat['1B'] = (bat['H'] - bat['2B'] - bat['3B'] - bat['HR'])

bat['SLG'] = (bat['1B'] + 2 * bat['2B'] + 3 * bat['3B'] + 4 * bat['HR']) / bat['AB']
bat['SLG'].fillna(0.0, inplace=True)
bat['OPS'] = bat['OBP'] + bat['SLG']


#def BA(H, AB):
#    return H / AB

#bat['BA'] = bat[['H', 'AB']].apply(lambda x: BA(x[0], x[1]), axis=1)


In [ ]:
# calculate the league leaders in 1B, 2B, 3B, 4B

bat.rank()
bat.groupby('yearID').rank()
bat.groupby('yearID').idxmax()

In [ ]:
# filter the dataset for AB qualifiers

bat = bat[bat['AB'] > 100]

bat.head(10)

In [ ]:
# join/merge the dataframe you created with df_player (on player id)
# calculate each players age in a given year (just use yearID - birthYear)
# group by age and then calculate the batting average and slugging percentage
# over all players of a given age
# at which age do players have the highest batting average and slugging percentage?

play = players.set_index('playerID')
bat.join(play)
bat.join(play, on='playerID')

bat.merge(play, on='playerID', how='inner') #what's the problem with this? no yearID
batting.merge(players, on='playerID')

bat_comb = bat.join(play, on='playerID')
bat_comb = bat_comb.reset_index()
bat_comb['age'] = bat_comb['yearID'] - bat_comb['birthYear']
bat_comb.set_index(['playerID', 'yearID'], inplace=True)

age_perf = bat_comb.groupby('age')[['BA', 'OBP', 'SLG', 'OPS']].mean()



In [ ]:
### join the salary table with the player table and the table you made
### where you calculated OBP and SLG, etc. (WHAT KIND OF JOIN DO YOU WANT TO DO)
### calculate the total runs scored by a player of a given age
### for every year, what is average cost of a run scored for each age?

money['yearID']
money['yearID'].min()
money.isnull().sum()
money.duplicated().sum()
mon_grp = money.groupby(['playerID', 'yearID']).sum()

bat_comb.join(mon_grp)
mon_grp = bat_comb.join(mon_grp, how='inner')

mon_grp.groupby('age')[['R', 'salary']].sum().apply(lambda x: x[1] / x[0], axis=1)

In [ ]:
# assign a player a position for a year based on which position they played the most

pos.head(10)
pos.columns

pos_comb = pos.groupby(['playerID', 'yearID']).sum()
pos_comb.idxmax(axis=1)
pos_comb = pos_comb.drop(columns=['G_all', 'GS', 'G_batting', 'G_defense'])
pos_comb['pos'] = pos_comb.idxmax(axis=1).str.split('_').apply(lambda x: x[1])

In [ ]:
### stack to long-format data
pos_long = pos_comb.stack()
pos_long = pos_long[pos_long > 0]

pos_long.unstack().fillna(0.0)

pos_comb.reset_index().melt(id_vars=['playerID', 'yearID'], value_vars = ['G_1b', 'G_2b'])

In [ ]:
mon_grp.join(pos_comb).groupby(['yearID', 'pos'])['salary'].mean().groupby('yearID').idxmax()

In [ ]:
### combine outfield into one category

positions = pos_comb['pos'].unique()
pos_map = {pos: 'of' if pos in ['cf', 'rf', 'lf'] else pos for pos in positions}
pos_comb['pos_cons'] = pos_comb['pos'].map(pos_map)

In [ ]:
mon_grp.join(pos_comb).groupby(['yearID', 'pos_cons'])['salary'].mean().groupby('yearID').idxmax()

In [ ]:
### bucket players by weight
### calculate the career number of games played by weight

players['weight_class'] = pd.qcut(players['weight'], q=10)

bt_wt = bat.copy()

bt_wt = bt_wt.join(players.set_index('playerID')['weight_class'])
bt_wt = bt_wt.dropna(how='any')

bt_wt.groupby('weight_class')['HR'].mean()

In [ ]:
bt_wt.query('yearID > 1990')['weight_class'].sort_values()